## Drug-Vascular Distribution Analysis Notebook
This Jupyter Notebook presents a novel methodology to analyze the distribution of drugs in relation to vascular structures in microscopy images.  
The core idea is to process given images of drug distribution and vascular structures to mask vascular regions either with or without drug presence. Then, calculated the vascular related drug distribution distances.  
Visulaizing parts are also included.  

Author: takakiom  
Date: 09/19/2023  
License: MIT License (refer to the LICENSE file in the root directory for more details)

**Main Functions:**  
**1. read_and_process_images:** Reads and processes the drug and vascular images. The drug areas with zero values are masked on the vascular image.  
**2. threshold_images:** Applies a binary threshold to the drug and vascular images.  
**3. find_contours:** Finds the contours in the thresholded drug image and filters them based on area.  
**4. analyze_contours:** Analyzes the contours to draw lines from drug regions to the nearest vascular regions.  
**5. is_valid_line:** Checks if a line between two points lies entirely within the drug area.  
**6. find_nearest_valid_point:** Finds the nearest vascular point to a given drug point such that the line between them lies entirely within the drug area.  
**7. read_and_process_stacks:** Reads and processes stacks of drug and vascular images.  
**8. process_image_pair:** Processes a pair of drug and vascular images.  
**9. create_composite_image:** Combines the binary drug and vascular images into a composite color image.  
**10. draw_and_save_lines:** Draws the lines found during contour analysis on the original image.  
**11. draw_and_save_all_lines:** Draws all the lines for each slice of the image stack.  
**12. mean_distance:** Calculates the mean distance from drug regions to vascular regions for each slice and saves it.  
**13. process_image:** Processes each slice of the image stack.

In [ ]:
import cv2
import numpy as np
from scipy.spatial import cKDTree as KDTree
import matplotlib.pyplot as plt
from bresenham import bresenham
from tifffile import imwrite, imread
import os
import seaborn as sns
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Constants
# These are predefined values that are used throughout the code. Change them as appropriate for your data.
THRESHOLD_VALUE = 127
MAX_VALUE = 255
CONTOUR_AREA_THRESHOLD = 4
K_NEAREST = 10
THRESHOLD = 0.05
LINE_WIDTH = 1
DPI = 900 

def read_and_process_images(drug_path, vascular_path):
    img_drug = cv2.imread(drug_path, cv2.IMREAD_GRAYSCALE)
    img_vasc = cv2.imread(vascular_path, cv2.IMREAD_GRAYSCALE)
    mask_zero_drug = img_drug == 0
    img_vasc[mask_zero_drug] = 0
    return img_drug, img_vasc

def threshold_images(img_drug, img_vasc):
    _, img_drug_bin = cv2.threshold(img_drug, THRESHOLD_VALUE, MAX_VALUE, cv2.THRESH_BINARY)
    _, img_vasc_bin = cv2.threshold(img_vasc, THRESHOLD_VALUE, MAX_VALUE, cv2.THRESH_BINARY)
    return img_drug_bin, img_vasc_bin

def find_contours(img_drug_bin):
    contours, _ = cv2.findContours(img_drug_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    parent_contours = [contour for contour in contours if cv2.contourArea(contour) > CONTOUR_AREA_THRESHOLD]
    child_contours = [contour for contour in contours if cv2.contourArea(contour) < CONTOUR_AREA_THRESHOLD]
    filtered_child_contours = [contour for contour in child_contours if cv2.contourArea(contour) >= CONTOUR_AREA_THRESHOLD]
    return parent_contours, filtered_child_contours

def analyze_contours(parent_contours, filtered_child_contours, img_drug_bin, vascular_points_array, ball_tree):
    lines = []
    line_lengths = []
    dist_img = np.zeros_like(img_drug_bin, dtype=np.float32)
    count_img = np.zeros_like(img_drug_bin, dtype=np.uint32)
    for contour in parent_contours + filtered_child_contours:
        contour_points_array = np.squeeze(contour)
        for contour_point in contour_points_array:
            vascular_point = find_nearest_valid_point(ball_tree, contour_point, img_drug_bin, vascular_points_array)
            if vascular_point is not None:
                line_segment = (contour_point[0], vascular_point[0], contour_point[1], vascular_point[1])
                lines.append(line_segment)
                line_length = np.linalg.norm(np.array(contour_point) - np.array(vascular_point))
                line_lengths.append(line_length)
                dist_img[vascular_point[1], vascular_point[0]] += line_length
                count_img[vascular_point[1], vascular_point[0]] += 1
    return lines, line_lengths, dist_img, count_img

def is_valid_line(img, start_point, end_point):
    x0, y0 = map(int, start_point)
    x1, y1 = map(int, end_point)
    points = list(bresenham(x0, y0, x1, y1))
    return all(img[y, x] != 0 for x, y in points)


# Function to find the nearest valid point with the constraint
def find_nearest_valid_point(ball_tree, contour_point, img, vascular_points_array):
    distances, indices = ball_tree.query(contour_point.reshape(1, -1), k=10)
    for index in indices[0]:
        vascular_point = vascular_points_array[index]
        if is_valid_line(img, tuple(contour_point), tuple(vascular_point)):
            return vascular_point
    return None

def read_and_process_stacks(drug_path, vascular_path):
    img_drug_stack = imread(drug_path)
    img_vasc_stack = imread(vascular_path)
    return img_drug_stack, img_vasc_stack

def process_image_pair(img_drug, img_vasc):
    img_drug_bin, img_vasc_bin = threshold_images(img_drug, img_vasc)
    parent_contours, filtered_child_contours = find_contours(img_drug_bin)
    vascular_points_y, vascular_points_x = np.where(img_vasc_bin > 0)
    vascular_points_array = np.column_stack([vascular_points_x, vascular_points_y])
    ball_tree = KDTree(vascular_points_array)
    results = analyze_contours(parent_contours, filtered_child_contours, img_drug_bin, vascular_points_array, ball_tree)
    return results

def create_composite_image(img_drug_bin, img_vasc_bin):
    # For Magenta: [255, 0, 255] (img_drug_bin)
    red_channel_drug = img_drug_bin * 255
    green_channel_drug = img_drug_bin * 0
    blue_channel_drug = img_drug_bin * 255
    
    # For Yellow: [255, 255, 0] (img_vasc_bin)
    red_channel_vasc = img_vasc_bin * 255
    green_channel_vasc = img_vasc_bin * 255
    blue_channel_vasc = img_vasc_bin * 0
    
    # Combine them into a 3D array
    red_channel = red_channel_drug + red_channel_vasc
    green_channel = green_channel_drug + green_channel_vasc
    blue_channel = blue_channel_drug + blue_channel_vasc
    
    # Stack them to form the color image
    color_img = np.stack([red_channel, green_channel, blue_channel], axis=-1)
    
    # Normalize to 8-bit (0-255) if needed
    color_img = (color_img * 255 / color_img.max()).astype(np.uint8)
    
    return color_img

def draw_and_save_lines(lines_with_index, img_drug, img_vasc_bin, index, color, ax, output_image, threshold=THRESHOLD):
    for line_with_index in lines_with_index:
        line, line_index = line_with_index  
        if line_index == index:  
            if color == 'cyan':
                cv2.line(output_image, (line[0], line[2]), (line[1], line[3]), 255, LINE_WIDTH)
                ax.plot([line[0], line[1]], [line[2], line[3]], color=color, alpha=1, linewidth=0.5)
            else:
                cv2.line(output_image, (line[0], line[2]), (line[1], line[3]), 255, LINE_WIDTH)
                ax.plot([line[0], line[1]], [line[2], line[3]], color='gray', alpha=1, linewidth=0.5)

def draw_and_save_all_lines(top_lines_with_index, remaining_lines_with_index, img_drug, img_vasc_bin, index, img_drug_path, threshold=THRESHOLD):
    # Create directories for saving images if they don't exist
    line_path = os.path.join(os.path.dirname(img_drug_path), f'top{THRESHOLD*100}%_lines')
    line_tif = os.path.join(line_path, 'tif')
    line_png = os.path.join(line_path, 'png')
    os.makedirs(line_path, exist_ok=True)
    os.makedirs(line_tif, exist_ok=True)
    os.makedirs(line_png, exist_ok=True)
    
    # Binary image for TIFF
    output_image = np.zeros_like(img_drug)
    
    # Open a new plot here for each slice
    fig, ax = plt.subplots()
    color_img = create_composite_image(img_drug, img_vasc_bin)
    ax.imshow(color_img)

    # Draw red lines
    draw_and_save_lines(top_lines_with_index, img_drug, img_vasc_bin, index, color='cyan', ax=ax, output_image=output_image)

    # Draw gray lines
    draw_and_save_lines(remaining_lines_with_index, img_drug, img_vasc_bin, index, color='gray', ax=ax, output_image=output_image)

    # Save the combined TIFF and PNG images
    ax.axis('off')
    plt.savefig(os.path.join(line_png, f'Combined_Top{threshold*100}%_{index}.png'), bbox_inches='tight', pad_inches=0, dpi=DPI)
    plt.savefig(os.path.join(line_tif, f'Combined_Top{threshold*100}%_{index}.tif'), bbox_inches='tight', pad_inches=0, dpi=DPI, format='tif')
    plt.close()



def mean_distance(dist_img, count_img, index, img_drug_path, threshold=THRESHOLD):
    # saving paths
    img_path = os.path.join(os.path.dirname(img_drug_path), f'top{THRESHOLD*100}%_mean_distances')
    mean_path = os.path.join(img_path, 'raw_values')
    mean_binary = os.path.join(img_path, 'binarized')
    os.makedirs(img_path, exist_ok=True)
    os.makedirs(mean_path, exist_ok=True)
    os.makedirs(mean_binary, exist_ok=True)
    mean_dist_img = np.divide(dist_img, count_img, out=np.zeros_like(dist_img), where=count_img != 0)
    # Convert the image to 8-bit unsigned integer
    mean_dist_img_8bit = mean_dist_img.astype(np.uint8)
    # Save as a TIFF file
    mean_img = os.path.join(mean_path, f'mean_distance_{index}.tiff')
    imwrite(mean_img, mean_dist_img_8bit)
    # binarized
    binarized_image =  np.where(mean_dist_img_8bit > 0.5, 255, 0)
    binarized_image = binarized_image.astype(np.uint8)
    binary_img = os.path.join(mean_binary, f'mean_distance_binary_{index}.tiff')
    imwrite(binary_img, binarized_image)

def process_image(i, img_drug_stack, img_vasc_stack, img_drug_path):
    img_drug, img_vasc = img_drug_stack[i], img_vasc_stack[i]
    img_drug_bin, img_vasc_bin = threshold_images(img_drug, img_vasc)
    results = process_image_pair(img_drug, img_vasc)
    mean_distance(results[2], results[3], i, img_drug_path)
    return i, results

img_drug_path = "/pre-processed/binarized/tumor/channel/image.tif"
vascular_image_path = "/pre-processed/binarized/vascular/channel/image.tif"
img_drug_stack, img_vasc_stack = read_and_process_stacks(img_drug_path, vascular_image_path)

# Aggregated results
all_results = []

# Sum of all results
sum_dist_img = np.zeros_like(img_drug_stack[0], dtype=np.float32)
sum_count_img = np.zeros_like(img_drug_stack[0], dtype=np.uint32)

for i in range(len(img_drug_stack)):
    img_drug, img_vasc = img_drug_stack[i], img_vasc_stack[i]
    img_drug_bin, img_vasc_bin = threshold_images(img_drug, img_vasc)
    results = process_image_pair(img_drug, img_vasc)
    all_results.append((i, results))

    # Update the sum of the results
    _, _, dist_img, count_img = results
    mean_distance(dist_img, count_img, i, img_drug_path)
    sum_dist_img += dist_img
    sum_count_img += count_img
    
# Accumulate Lines and Lengths
all_lines = []
all_line_lengths = []
for image_index, results in all_results:
    lines, line_lengths, _, _ = results
    lines_with_index = [(line, image_index) for line in lines]  # Include image index
    all_lines.extend(lines_with_index)
    all_line_lengths.extend(line_lengths)

# Modification in Sorting and Selecting Lines
sorted_lines_with_index = [line for _, line in sorted(zip(all_line_lengths, all_lines), key=lambda pair: pair[0], reverse=True)]
num_top_lines = int(THRESHOLD * len(sorted_lines_with_index))
top_lines_with_index = sorted_lines_with_index[:num_top_lines]
# New line: Selecting the remaining 95% of lines
remaining_lines_with_index = sorted_lines_with_index[num_top_lines:]

# Drawing Both Sets of Lines
for i in range(len(img_drug_stack)):
    print(f'Start processing slice number: {i}')
    img_drug, img_vasc = img_drug_stack[i], img_vasc_stack[i]
    img_drug_bin, img_vasc_bin = threshold_images(img_drug, img_vasc)
    
    draw_and_save_all_lines(top_lines_with_index, remaining_lines_with_index, img_drug, img_vasc_bin, i, img_drug_path)
    print('- fin')


## Plots

In [ ]:
# Compute the mean distance where the count is not zero
mean_dist_img = np.divide(sum_dist_img, sum_count_img, out=np.zeros_like(sum_dist_img), where=sum_count_img != 0)

# Flatten the mean distance image to a 1D array and remove zeros
distances = mean_dist_img.flatten()
distances = distances[distances != 0]

def accumulate_and_sort_lines(all_results, threshold):
    all_lines = []
    all_line_lengths = []
    for image_index, results in all_results:
        lines, line_lengths, _, _ = results
        lines_with_index = [(line, image_index) for line in lines]
        all_lines.extend(lines_with_index)
        all_line_lengths.extend(line_lengths)

    sorted_lines_with_index = [line for _, line in sorted(zip(all_line_lengths, all_lines), key=lambda pair: pair[0], reverse=True)]
    num_top_lines = int(threshold * len(sorted_lines_with_index))
    return sorted_lines_with_index[:num_top_lines], all_line_lengths

def extract_lengths(sorted_lines_with_index, all_line_lengths, threshold):
    num_top_lines = int(threshold * len(sorted_lines_with_index))
    top_line_lengths = [length for length, line in sorted(zip(all_line_lengths, all_lines), key=lambda pair: pair[0], reverse=True)[:num_top_lines]]
    other_line_lengths = [length for length, line in sorted(zip(all_line_lengths, all_lines), key=lambda pair: pair[0], reverse=True)[num_top_lines:]]
    return top_line_lengths, other_line_lengths

def plot_histogram(distances, top_line_lengths, other_line_lengths, threshold):
    if len(distances) > 0:
        num_unique_distances = np.unique(distances).shape[0]
        plt.hist(other_line_lengths, bins=num_unique_distances, color='gray', edgecolor='gray', log=True, label='Other lines')
        plt.hist(top_line_lengths, bins=num_unique_distances, color='cyan', edgecolor='cyan', log=True, label=f'Top {threshold*100:.2f}% longest lines')
        plt.title('Histogram of Distances')
        plt.xlabel('Distance [pxl = 3.5 um]')
        plt.ylabel('Number of Points (Log Scale)')
        plt.legend()
        plt.grid(True)
        plt.savefig(f'Histogram_log-scale.png', dpi=900, bbox_inches='tight')
        plt.show()
    else:
        print("No distances were computed.")
        
def plot_histogram_linear_scale(top_line_lengths, other_line_lengths, threshold):
    num_unique_distances = np.unique(top_line_lengths + other_line_lengths).shape[0]
    plt.hist(other_line_lengths, bins=num_unique_distances, color='gray', edgecolor='gray', label='Other lines')
    plt.hist(top_line_lengths, bins=num_unique_distances, color='cyan', edgecolor='cyan', label=f'Top {threshold*100:.2f}% longest lines')
    #plt.title('Histogram of Distances')
    plt.xlabel('Distance [pxl = 3.5 um]')
    plt.ylabel('Number of Points')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'Histogram_linear_scale.png', dpi=900, bbox_inches='tight')
    plt.show()

def plot_box_plot(top_line_lengths, other_line_lengths):
    # Combine top_line_lengths and other_line_lengths into a single array
    line_lengths = top_line_lengths + other_line_lengths
    labels = ['Top Lines'] * len(top_line_lengths) + ['Other Lines'] * len(other_line_lengths)
    
    # Create a DataFrame for plotting
    df = pd.DataFrame({'Lengths': line_lengths, 'Type': labels})
    
    # Define the color palette
    custom_palette = {'Top Lines': 'cyan', 'Other Lines': 'gray'}
    
    # Plot the box plot without individual data points
    sns.boxplot(x='Type', y='Lengths', data=df, palette=custom_palette)
    
    plt.title('Box Plot of Line Lengths')
    plt.ylabel('Length [pxl = 3.5 um]')
    plt.xlabel('Category')
    
    # Save the plot
    plt.savefig('Box_plot.png', dpi=900, bbox_inches='tight')
    plt.show()

# main

sorted_lines_with_index, all_line_lengths = accumulate_and_sort_lines(all_results, THRESHOLD)
top_line_lengths, other_line_lengths = extract_lengths(sorted_lines_with_index, all_line_lengths, THRESHOLD)

plot_histogram(distances, top_line_lengths, other_line_lengths, THRESHOLD)
plot_histogram_linear_scale(top_line_lengths, other_line_lengths, THRESHOLD)
plot_box_plot(top_line_lengths, other_line_lengths)



## Drug related vascular image, and Vascular area with no drug

- 'no_drug_vascular' means vascular area with drug distribution NOT related
- 'with_drug_vascular' means vascular area with drug distribution

In [ ]:
def process_and_save_images(img_drug_stack, img_vasc_stack, no_drug_path, with_drug_path):
    for index, (img_drug, img_vasc) in enumerate(zip(img_drug_stack, img_vasc_stack)):
        no_drug_img = mask_vasc_with_drug_zero(img_drug.copy(), img_vasc.copy())
        with_drug_img = mask_vasc_with_drug_255(img_drug.copy(), img_vasc.copy())

        # Define the output paths using the index to create unique filenames
        output_path_no_drug = os.path.join(no_drug_path, f'no_drug_vascular_{index}.tif')
        output_path_with_drug = os.path.join(with_drug_path, f'with_drug_vascular_{index}.tif')

        # Save the images
        cv2.imwrite(output_path_no_drug, no_drug_img)
        cv2.imwrite(output_path_with_drug, with_drug_img)

def mask_vasc_with_drug_zero(img_drug, img_vasc):
    mask_zero_drug = img_drug == 0
    img_vasc[mask_zero_drug] = 0
    _, img_vasc_bin = cv2.threshold(img_vasc, 127, 255, cv2.THRESH_BINARY)
    return img_vasc_bin

def mask_vasc_with_drug_255(img_drug, img_vasc):
    mask_zero_drug = img_drug == 255
    img_vasc[mask_zero_drug] = 0
    _, img_vasc_bin = cv2.threshold(img_vasc, 127, 255, cv2.THRESH_BINARY)
    return img_vasc_bin

img_drug_stack, img_vasc_stack = read_and_process_stacks(img_drug_path, vascular_image_path)

vasc_drug_path = os.path.join(os.path.dirname(img_drug_path), 'drug_related_vasc')
no_drug_path = os.path.join(vasc_drug_path, 'no_drug')
with_drug_path = os.path.join(vasc_drug_path, 'with_drug')

# Create the directories if they don't exist
os.makedirs(no_drug_path, exist_ok=True)
os.makedirs(with_drug_path, exist_ok=True)

process_and_save_images(img_drug_stack, img_vasc_stack, no_drug_path, with_drug_path)


## Visualizing the distribution through slices

In [ ]:
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Get all image paths in the './test' directory with the extension '.tiff'
image_paths = glob.glob('/the/directory/automatically/generated/top5.0%_mean_distances/raw_values/*.tiff')

# Initialize a list to store the pixel values greater than 0 for each image
pixels_gt_zero_values = []

# Iterate through the images
for image_path in image_paths:
    # Load the image
    image = Image.open(image_path)
    
    # Convert to a numpy array
    image_array = np.array(image)
    
    # Find the pixels with values greater than 0
    pixels_gt_zero = image_array[image_array > 0]
    
    # Append the values to the list
    pixels_gt_zero_values.append(pixels_gt_zero)

# Plot the box plots for each image
plt.figure(figsize=(12, 6)) # Adjust the figure size
boxplot = plt.boxplot(
    pixels_gt_zero_values, 
    labels=[f'{i+1}' for i in range(len(image_paths))], 
    flierprops={'markerfacecolor': 'b', 'markersize': 1}
)
# Change the style of the median lines
for median in boxplot['medians']:
    median.set(color='red', linewidth=3)
plt.ylabel('Distances [pxl = 3.5 um]')
plt.title('Distribution of Vascular-Drug distances')
plt.xticks(rotation=45, fontsize=8) # Rotate x-axis labels
plt.tight_layout() # Ensure that everything fits without overlapping
plt.savefig('./Boxplot_for_each_slice.png', dpi=300) # Save the figure with desired resolution
plt.show()


## Remarks:
This code performs a thorough analysis of the distribution of drugs in relation to vascular structures in microscopy images.  
By analyzing the distance between drug regions and vascular structures, it provides insights into the efficiency of drug delivery.  
Always ensure to adjust the constants and paths according to the specific dataset and requirements.  
It's crucial to note that the efficiency of this code might vary depending on the quality and type of the images being processed.   
Always validate the results using a representative sample before full-scale processing.